In [9]:
from utils.EDA.analysis import *
from utils.EDA.visualize import *

In [10]:
import statsmodels.api as sm

In [11]:
def LinearReg(data, target, *features):
    y = np.array(data[target])[np.newaxis].T

    X = np.ones([len(data), 1])
    for feature in features:
        X = np.hstack((X, np.array(data[feature])[np.newaxis].T))

    w, resid, rank, s = np.linalg.lstsq(X, y)

    print("w{} : {}".format("Const", w[0]), end = " ")
    
    for i in range(len(w)-1):
        print("w{} : {}".format(features[i], w[i+1]), end = " ")
        
    RMSE = np.sqrt(resid / len(data))
    print("\n RMSE : {}".format(RMSE))
    print("rank : {}".format(rank))
    
    print("cross validation : {}".format(crossValidation(10, data, target, *features)))
    
    # draw chart
    # 예측치와 실제치과의 관계표시(scatter plot)
    predict = np.dot(X, w)
    
    plt.scatter(y, predict)
    plt.xlabel("target value")
    plt.ylabel("predict value")
    plt.xlim(data[target].min(), data[target].max())
    plt.show()

In [12]:
def crossValidation(num, data, target, *features):
    rmses = []
    
    # 데이터프래임에서 target과 features 추출 
    data = data[[*features, target]]
    
    # 데이터프래임 num 갯수로 분할
    remain = data.shape[0] % num
    data_set = np.split(data.values[:data.shape[0]-remain], num)
    data_set[num-1] = np.vstack([data_set[num-1], data.values[data.shape[0]-remain:]])
    
    # 각 fold로 계산하여 rmse를 계산
    for i in range(num):
        
        # test_set과 train_set 제작
        test_set = data_set[i] # test set 

        if (i==0): # train set
            train_set = data_set[1:]
        elif (i==9):
            train_set = data_set[:num-1]
        else:
            train_set = data_set[:i]
            train_set += data_set[i+1:]
        train_set = np.concatenate(train_set)
        
        # train_set으로 linear regression 모델 훈련
        y = train_set[:, -1][np.newaxis].T
        X = np.ones([train_set.shape[0], 1])
        X = np.hstack((X, train_set[:, :train_set.shape[1]-1]))
        
        w, resid, rank, s = np.linalg.lstsq(X, y)
        
        # test_set으로 predict 계산 및 target 생성
        test_X = np.hstack((np.ones([test_set.shape[0], 1]), test_set[:, :test_set.shape[1]-1]))
        predict_test = np.dot(test_X, w).T[0]
        target_test = test_set[:,-1]
        
        # rmse 계산
        rss = np.square(predict_test - target_test).sum()
        rmses.append(np.sqrt(rss / test_X.shape[0]))    
    
    # rmse_avr 계산
    rmse_avr = sum(rmses) / num
    
    # print rmse
#     for idx, rmse in enumerate(rmses):
#         print("test{} : {:1.2f}".format(idx, rmse), end = " ")
    
#     print("\n average of rmse : {:1.2f}".format(rmse_avr))
    return rmse_avr
        

In [13]:
def LinearRegSM(data, target, *features):
    y = np.array(data[target])[np.newaxis].T
    
    #data[*features]
    dfX = sm.add_constant(data[[*features]])
    dfy = data[target]
    
    model = sm.OLS(dfy, dfX)
    result = model.fit()
    print(result.summary())
    return result

In [14]:
plt.style.use("ggplot")

In [15]:
df_train = pd.read_csv("../data/train_no_missing.csv")
df_test = pd.read_csv("../data/test.csv")

# LinearRegression

523, 1298

### 실험

In [16]:
get_top_abs_cor(df_train[top_cor], 10)

NameError: name 'top_cor' is not defined

연관이 있는 값인 (GarageCars, GarageArea), (GrLivArea, TotRmsAbvGrd), (TotalBsmtSF, 1stFlrSF) 에서 "GarageArea", "TotRmsAbvGrd", "1stFlrSF" 제거
37686 -> 37591 로 감소

# Logarithm

SalePrice

In [ ]:
log_SalePrice = np.log(df_train["SalePrice"])
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

sp.stats.probplot(df_train["SalePrice"], plot = axes[0])
sp.stats.probplot(log_SalePrice, plot = axes[1])

LotFrontage(선택)

In [ ]:
log_Frontage = np.log(df_train["LotFrontage"])
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

sp.stats.probplot(df_train["LotFrontage"], plot = axes[0])
sp.stats.probplot(log_Frontage, plot = axes[1])

LotArea(선택)

In [ ]:
log_LotArea = np.log(df_train["LotArea"])
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

sp.stats.probplot(df_train["LotArea"], plot = axes[0])
sp.stats.probplot(log_LotArea, plot = axes[1])

1stFlrSF(선택)

In [ ]:
log_1stFlrSF = np.log(df_train["FirstFlrSF"])
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

sp.stats.probplot(df_train["FirstFlrSF"], plot = axes[0])
sp.stats.probplot(log_1stFlrSF, plot = axes[1])

GrLivArea(선택)

In [ ]:
log_GrLivArea = np.log(df_train["GrLivArea"])
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

sp.stats.probplot(df_train["GrLivArea"], plot = axes[0])
sp.stats.probplot(log_GrLivArea, plot = axes[1])

In [ ]:
list(top_cor)

In [ ]:
# log 사용(SalePrice, GrLivArea)
result = LinearReg(df_new_train, "log_SalePrice", 
                   'OverallQual',
                   'log_GrLivArea',
                   'GarageArea',
                   '1stFlrSF',
                   'FullBath',
                   'YearBuilt',
                   'YearRemodAdd')
# 0.17 -> 0.16

# make new features about SF

In [ ]:
df_train.filter(regex = "SF").tail()

In [ ]:
# make Area

In [ ]:
df_new_train["TotalArea"] = df_train[["1stFlrSF", "2ndFlrSF", "TotalBsmtSF", "LowQualFinSF"]].sum(axis = 1)

In [ ]:
sum(df_new_train["TotalArea"] == 0)

In [ ]:
# make porch Area

In [ ]:
df_new_train.filter(regex = "Porch").columns

In [ ]:
df_new_train["PorchArea"] = df_new_train[["OpenPorchSF", "EnclosedPorch", "3SsnPorch", "ScreenPorch"]].sum(axis = 1)

In [ ]:
sum(df_new_train["PorchArea"] == 0)

In [ ]:
# salesPrice와의 상관관계

In [ ]:
df_new_train[["SalePrice", "TotalArea", "PorchArea"]].corr()

In [ ]:
# TotalArea는 1SF보다 더 상관관계가 많다. 그래서 TotalArea 추가
cor = df_train[df_new_train].corr()
cor["GrLivArea"].sort_values(ascending = False)[:10]

In [ ]:
# totalArea 분석

In [ ]:
sns.distplot(df_new_train["TotalArea"])
df_new_train["TotalArea"].skew()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

sp.stats.probplot(df_new_train["TotalArea"], plot = axes[0])
sp.stats.probplot(np.log(df_new_train["TotalArea"]), plot = axes[1])
# 사용하지 않음

In [ ]:
cor_top = df_new_train[["log_SalePrice",
              'OverallQual',
              'log_GrLivArea',
              'GarageArea',
              '1stFlrSF',
              'FullBath',
              'YearBuilt',
              'YearRemodAdd', "TotalArea"]].corr()

In [ ]:
cor_top["TotalArea"].sort_values(ascending = False)

In [ ]:
#log 사용(SalePrice, GrLivArea)
result = LinearReg(df_new_train, "log_SalePrice",
              'OverallQual',
              'log_GrLivArea',
              'GarageArea',
              'FullBath',
              'YearBuilt',
              'YearRemodAdd', 
              "TotalArea" )
# 0.167 -> 0.168

In [ ]:
#log 사용(SalePrice, GrLivArea)
result = LinearRegSM(df_new_train, "log_SalePrice",
              'OverallQual',
              'log_GrLivArea',
              'GarageArea',
              'FullBath',
              'YearBuilt',
              'YearRemodAdd', 
              "TotalArea",
                    *neibor)
# 0.167 -> 0.168

In [ ]:
neibor = df_new_train.filter(regex="Neighbor").columns

In [ ]:
#log 사용(SalePrice, GrLivArea)
result = LinearRegSM(df_new_train, 
                    "log_SalePrice", *neibor)
# 0.167 -> 0.168